In [ ]:
import requests
import json
import os


class RebyteAIClient:
    def __init__(
        self,
        base_url: str,
        provider_id: str = "openai",
        model_id: str = "gpt-3.5-turbo-1106",
        use_cache: bool = True,
        use_stream: bool = False,
        seed=None,
        response_format=None,
        version=1,
    ):
        self.base_url = base_url
        self.headers = {
            "Authorization": f"Bearer {os.getenv('REBYTE_API_KEY')}",
            "Content-Type": "application/json",
        }
        self.config = {
            "MODEL_CALL": {
                "provider_id": provider_id,
                "model_id": model_id,
                "use_cache": use_cache,
                "use_stream": use_stream,
                "seed": seed,
                "response_format": response_format,
            }
        }
        self.version = version

    def make_request(self, inputs: any):
        data = {
            "version": self.version,
            "config": self.config,
            "blocking": True,
            "inputs": inputs,
        }
        response = requests.post(
            self.base_url, headers=self.headers, data=json.dumps(data)
        )
        if response.status_code != 200:
            raise Exception(f"Http reauest error. code:{response.status_code}")
        result = response.json()
        traces = result["run"]["traces"]
        output = traces[len(traces) - 1]
        return (
            output[1][0][0]["value"]
            if output[1][0][0]["value"] != None
            else output[1][0][0]["error"]
        )


from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

client = RebyteAIClient(
    base_url="https://rebyte.ai/api/sdk/p/3f882a52c1d29c709ee0/a/74e717e8be9fbc415899/r",
    model_id="gpt-4-1106-preview",
    version=2,
)
response = client.make_request({"messages": [{"role": "user", "content": "btc price"}]})
print(response)

In [2]:
from __future__ import annotations

import json
from typing import Any, Dict, List, Optional, Tuple, Union

from langchain.schema.agent import AgentAction, AgentFinish
from langchain.schema.runnable import RunnableConfig, RunnableSerializable
from langchain.tools.base import BaseTool
from pydantic import BaseModel, Field


class OpenAIAssistantFinish(AgentFinish):
    """AgentFinish with run and thread metadata."""


class OpenAIAssistantAction(AgentAction):
    """AgentAction with info needed to submit custom tool output to existing run."""

    tool_call_id: str
    origin_args: str


class OpenAIFunctionAction(AgentAction):
    """AgentAction with info needed to submit custom tool output to existing run."""

    origin_args: str


OutputType = Union[
    List[OpenAIAssistantAction],
    OpenAIAssistantFinish,
]


def _get_openai_client() -> RebyteAIClient:
    return RebyteAIClient(
        base_url="https://rebyte.ai/api/sdk/p/3f882a52c1d29c709ee0/a/74e717e8be9fbc415899/r",
        model_id="gpt-4-1106-preview",
        version=2,
    )


class RebyteRunnable(BaseModel):
    client: RebyteAIClient = Field(default_factory=_get_openai_client)

    # def __init__(self, client: RebyteAIClient):
    #     self.client = client

    class Config:
        arbitrary_types_allowed = True

    def invoke(
        self, input: dict, config: Optional[RunnableConfig] = None
    ) -> OutputType:
        response = self.client.make_request(inputs={"messages": input})
        return self._get_response(response)

    def _parse_intermediate_steps(
        self, intermediate_steps: List[Tuple[OpenAIAssistantAction, str]]
    ) -> dict:
        tool_outputs = [
            {"output": output, "tool_call_id": action.tool_call_id}
            for action, output in intermediate_steps
        ]
        submit_tool_outputs = {
            "tool_outputs": tool_outputs,
        }
        return submit_tool_outputs

    def _get_response(self, response: dict) -> Any:
        # TODO: Pagination
        if (
            "function_call" not in response.keys()
            and "tool_calls" not in response.keys()
        ):
            return OpenAIAssistantFinish(
                return_values={"output": response["content"]},
                log="",
            )
        elif "tool_calls" in response.keys():
            actions = []
            for tool_call in response["tool_calls"]:
                function = tool_call["function"]
                args = json.loads(function.arguments)
                if len(args) == 1 and "__arg1" in args:
                    args = args["__arg1"]
                actions.append(
                    OpenAIAssistantAction(
                        tool=function.name,
                        tool_input=args,
                        tool_call_id=tool_call.id,
                        log="",
                        origin_args=function.arguments,
                    )
                )
            return actions
        elif "function_call" in response.keys():
            args = json.loads(response["function_call"]["arguments"])
            if len(args) == 1 and "__arg1" in args:
                args = args["__arg1"]
            action = OpenAIFunctionAction(
                tool=response["function_call"]["name"],
                tool_input=args,
                tool_call_id="",
                log="",
                origin_args=response["function_call"]["arguments"],
            )
            return action
        else:
            raise ValueError(f"Response Unexpected. {response}")


class AgentExecutor:
    chat_history: list[dict] = []
    agent: RebyteRunnable = None
    tools: list[BaseTool] = None

    def __init__(self, agent: RebyteRunnable, tools: list[BaseTool]):
        self.agent = agent
        self.tools = tools

    def execute_agent(self, input):
        tool_map = {
            tool.name: tool for tool in self.tools if isinstance(tool, BaseTool)
        }
        response = self.agent.invoke(input)
        for i in input:
            self.chat_history.append(i)
        while not isinstance(response, OpenAIAssistantFinish):
            if not isinstance(response, OpenAIFunctionAction):
                tool_outputs = []
                for action in response:
                    print(f"System: {action.tool} invoking.")
                    print(f"System: Input is {action.tool_input}")
                    self.chat_history.append(
                        {
                            "role": "tool",
                            "name": output["name"],
                            "tool_call_id": output["tool_call_id"],
                        }
                        for output in tool_outputs
                    )
                    try:
                        tool_output = tool_map[action.tool].invoke(action.tool_input)
                        # print(f"System: {action.tool} output {tool_output}")
                        tool_outputs.append(
                            {
                                "output": tool_output,
                                "tool_call_id": action.tool_call_id,
                                "name": action.tool,
                            }
                        )
                    except Exception as e:
                        tool_outputs.append(
                            {
                                "output": e,
                                "tool_call_id": action.tool_call_id,
                                "name": action.tool,
                            }
                        )
                self.chat_history.append(
                    {
                        "role": "tool",
                        "name": output["name"],
                        "content": output["output"],
                        "tool_call_id": output["tool_call_id"],
                    }
                    for output in tool_outputs
                )
                response = self.agent.invoke(input=self.chat_history)
            else:
                self.chat_history.append(
                    {
                        "role": "assistant",
                        "content": "",
                        "function_call": {
                            "name": response.tool,
                            "arguments": response.origin_args,
                        },
                    }
                )
                tool_output = tool_map[response.tool].invoke(response.tool_input)
                self.chat_history.append(
                    {
                        "role": "function",
                        "name": response.tool,
                        "content": tool_output,
                    }
                )
                response = self.agent.invoke(input=self.chat_history)
        self.chat_history.append(
            {"role": "assistant", "content": response.return_values["output"]}
        )
        return response

In [3]:
from langchain.agents import tool
from metaphor_python import Metaphor
import json
import os

metaphor = Metaphor(api_key=os.environ["METAPHOR_API_KEY"])

from langchain.agents import Tool
from openai_assistant_tools import GoogleSerperAPIWrapper
from langchain_openai import ChatOpenAI
from openai_assistant_tools import MyAPIChain

# from openai_assistant_api_doc import (
#     cmc_quote_lastest_api_doc,
#     cmc_trending_latest_api_doc,
#     cmc_trending_gainers_losers_api_doc,
#     cmc_trending_most_visited_api_doc,
#     cmc_metadata_api_doc,
# )
import openai_assistant_api_docs

newsSearch = GoogleSerperAPIWrapper(type="news")
# search = GoogleSearchAPIWrapper()
search = GoogleSerperAPIWrapper(type="search")
llm = ChatOpenAI(
    model="gpt-3.5-turbo-1106",
    verbose=True,
)
headers = {
    "Accepts": "application/json",
    "X-CMC_PRO_API_KEY": os.getenv("CMC_API_KEY"),
}
cmc_last_quote_api = MyAPIChain.from_llm_and_api_docs(
    llm=llm,
    api_docs=openai_assistant_api_docs.cmc_quote_lastest_api_doc,
    headers=headers,
    limit_to_domains=["https://pro-api.coinmarketcap.com"],
    verbose=True,
)
cmc_trending_latest_api = MyAPIChain.from_llm_and_api_docs(
    llm=llm,
    api_docs=openai_assistant_api_docs.cmc_trending_latest_api_doc,
    headers=headers,
    limit_to_domains=["https://pro-api.coinmarketcap.com"],
    verbose=True,
)
cmc_trending_gainers_losers_api = MyAPIChain.from_llm_and_api_docs(
    llm=llm,
    api_docs=openai_assistant_api_docs.cmc_trending_gainers_losers_api_doc,
    headers=headers,
    limit_to_domains=["https://pro-api.coinmarketcap.com"],
    verbose=True,
)
cmc_trending_most_visited_api = MyAPIChain.from_llm_and_api_docs(
    llm=llm,
    api_docs=openai_assistant_api_docs.cmc_trending_most_visited_api_doc,
    headers=headers,
    limit_to_domains=["https://pro-api.coinmarketcap.com"],
    verbose=True,
)
cmc_metadata_api = MyAPIChain.from_llm_and_api_docs(
    llm=llm,
    api_docs=openai_assistant_api_docs.cmc_metadata_api_doc,
    headers=headers,
    limit_to_domains=["https://pro-api.coinmarketcap.com"],
    verbose=True,
)

from openai_assistant_tools import TradingviewWrapper

tradingview = TradingviewWrapper(llm=llm)


@tool
def search(query: str, include_domains=None, start_published_date=None):
    """Search for a webpage based on the query.
    Set the optional include_domains (list[str]) parameter to restrict the search to a list of domains.
    Set the optional start_published_date (str) parameter to restrict the search to documents published after the date (YYYY-MM-DD).
    """
    r = []
    results = metaphor.search(
        f"{query}",
        use_autoprompt=True,
        num_results=10,
        include_domains=include_domains,
        start_published_date=start_published_date,
    )
    for rr in results.results:
        o = {
            "title": rr.title,
            "url": rr.url,
            "id": rr.id,
            "score": rr.score,
            "published_date": rr.published_date,
            "author": rr.author,
            "extract": rr.extract,
        }
        r.append(o)
    return json.dumps(r) if len(r) > 0 else f"There is no any result for query: {query}"


@tool
def find_similar(url: str):
    """Search for webpages similar to a given URL.
    The url passed in should be a URL returned from `search`.
    """
    r = []
    for rr in metaphor.find_similar(url, num_results=10).results:
        o = {
            "title": rr.title,
            "url": rr.url,
            "id": rr.id,
            "score": rr.score,
            "published_date": rr.published_date,
            "author": rr.author,
            "extract": rr.extract,
        }
        r.append(o)
    return json.dumps(r) if len(r) > 0 else f"There is no any result for url: {url}"


@tool
def get_contents(ids: list[str]):
    """Get the contents of a webpage.
    The ids passed in should be a list of ids returned from `search`.
    """
    r = []
    for rr in metaphor.get_contents(ids).contents:
        o = {
            # "id": rr.id,
            "url": rr.url,
            "title": rr.title,
            "extract": rr.extract,
            "author": rr.author,
        }
        r.append(o)
    return json.dumps(r) if len(r) > 0 else f"There is no any result for ids: {ids}"


tools = [
    {"type": "code_interpreter"},
    search,
    find_similar,
    get_contents,
    # Tool(
    #     name="SearchNews",
    #     func=newsSearch.run,
    #     description="""useful when you need search news about some terms. The input to this should be a some terms in English.""",
    #     coroutine=newsSearch.arun,
    # ),
    # Tool(
    #     name="GetContentFromURL",
    #     func=getHTMLFromURL,
    #     description="""useful when you need get the HTML of URL. The input to this should be URL returned from 'SearchNews'.""",
    # ),
    Tool(
        name="CryptocurrencyLatestQuote",
        func=cmc_last_quote_api.run,
        description="""useful when you need get a cryptocurrency's latest quote. The input to this should be a single cryptocurrency's symbol.""",
        coroutine=cmc_last_quote_api.arun,
    ),
    Tool(
        name="TrendingLatest",
        func=cmc_trending_latest_api.run,
        description="""useful when you need get a list of all trending cryptocurrency market data, determined and sorted by CoinMarketCap search volume. The input to this should be a complete question in English, and the question must have a ranking requirement, and the ranking cannot exceed 20.""",
        coroutine=cmc_trending_latest_api.arun,
    ),
    Tool(
        name="TrendingGainersAndLosers",
        func=cmc_trending_gainers_losers_api.run,
        description="""useful when you need get a list of all trending cryptocurrencies, determined and sorted by the largest price gains or losses. The input to this should be a complete question in English, and the question must have a ranking requirement, and the ranking cannot exceed 20.""",
        coroutine=cmc_trending_gainers_losers_api.arun,
    ),
    Tool(
        name="TrendingMostVisited",
        func=cmc_trending_most_visited_api.run,
        description="""useful when you need get a list of all trending cryptocurrency market data, determined and sorted by traffic to coin detail pages. The input to this should be a complete question in English, and the question must have a ranking requirement, and the ranking cannot exceed 20.""",
        coroutine=cmc_trending_most_visited_api.arun,
    ),
    Tool(
        name="MetaDataOfCryptocurrency",
        func=cmc_metadata_api.run,
        description="""useful when you need get all static metadata available for one or more cryptocurrencies. This information includes details like logo, description, official website URL, social links, and links to a cryptocurrency's technical documentation. The input to this should be a complete question in English.""",
        coroutine=cmc_metadata_api.arun,
    ),
    Tool(
        name="BuyOrSellSignal",
        func=tradingview.buySellSignal,
        description="""Useful when you need to know buy and sell signals for a cryptocurrency. The input to this should be a cryptocurrency's symbol.""",
        coroutine=tradingview.abuySellSignal,
    ),
]

In [4]:
import traceback

agent = RebyteRunnable(client=client)
question = "btc最近的行情如何"
print(f"You:{question}")
print("Loading...")
try:
    executor = AgentExecutor(agent=agent, tools=tools)
    output = executor.execute_agent(
        input=[{"role": "user", "content": question}],
    )
    print(output)
except Exception as e:
    print("Exception occurred: ", e)
    print("The traceback is:")
    traceback.print_exc()

You:btc最近的行情如何
Loading...


> Entering new MyAPIChain chain...
https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest?symbol=BTC
{"status":{"timestamp":"2024-01-17T01:49:14.668Z","error_code":0,"error_message":null,"elapsed":18,"credit_count":1,"notice":null},"data":{"BTC":[{"id":1,"name":"Bitcoin","symbol":"BTC","slug":"bitcoin","num_market_pairs":10712,"date_added":"2010-07-13T00:00:00.000Z","tags":[{"slug":"mineable","name":"Mineable","category":"OTHERS"},{"slug":"pow","name":"PoW","category":"ALGORITHM"},{"slug":"sha-256","name":"SHA-256","category":"ALGORITHM"},{"slug":"store-of-value","name":"Store Of Value","category":"CATEGORY"},{"slug":"state-channel","name":"State Channel","category":"CATEGORY"},{"slug":"coinbase-ventures-portfolio","name":"Coinbase Ventures Portfolio","category":"CATEGORY"},{"slug":"three-arrows-capital-portfolio","name":"Three Arrows Capital Portfolio","category":"CATEGORY"},{"slug":"polychain-capital-portfolio","name":"Polychain Capital Portfoli